In [1]:
from transformers import pipeline
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import sys
sys.path.append("../CODE-Baseline")  
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', None)  # 不截断列内容
pd.set_option('display.max_columns', None)   # 显示所有列
pd.set_option('display.max_rows', None)      # 显示所有行

file_path = '../DATASETS/salary_labelled_development_set.csv'
ignore_id_path = '../DATASETS/err_salary_develpment.csv'
df = pd.read_csv(file_path)
df_ignore = pd.read_csv(ignore_id_path)

print(df[df["job_id"] == 71382196])

# some y_true is impossible or wrong 
df = df[~df['job_id'].isin(df_ignore['job_id'])]
# print(df.head)

        job_id                             job_title  \
2107  71382196  【新人獎金可達$2,000】最新開幕_啟德AIRSIDE麥當勞_有薪培訓   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [8]:
country_currency_map = {
    "PH": "PHP", "AUS": "AUD", "NZ": "NZD", "SG": "SGD",
    "MY": "MYR", "TH": "THB", "ID": "IDR", "HK": "HKD"
}
salary_keywords = ['待遇', 'salary', 'wage', 'compensation', 'remuneration', 'gaji', 'bermula', 'basic', 'pokok', 'income']

def convert_k_to_number(text):
    # 匹配形如 20k、16.5k、30K 的数字
    def replace(match):
        num = float(match.group(1))
        return str(int(num * 1000))

    return re.sub(r'(\d+(?:\.\d+)?)k', replace, text, flags=re.IGNORECASE)
# html -> text
def clean_html_tags(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    for tag in soup(["script", "style"]):
        tag.decompose()
    text = soup.get_text(separator="\n", strip=True)
    
    # formate text
    text = re.sub(r"[•●▪►◆★♦✓✔⬤❖]", "", text)
    text = text.replace(",", "")
    for _, value in country_currency_map.items():
      text = text.replace(value, "$")
    text = text.replace("RM", "$")
    text = text.replace("฿", "$")
    text = text.replace("AU", "$")
    text = text.replace("$$", "$")
    text = text.replace("  ", " ")
    text = re.sub(r'\b[Tt][Oo]\b', '-', text)
    text = text.replace("and", "-")
    text = text.replace("至", "-")
    text = text.replace("hingga ke", "-")
    text = text.replace("hingga", "-")
    text = text.replace("Hingga", "-")
    text = text.replace("HINGGA", "-")
    text = convert_k_to_number(text)
    # 构建正则表达式，\b表示单词边界，|表示“或”
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in salary_keywords) + r')\b'

    # 替换为 compensation，flags=re.IGNORECASE 表示不区分大小写
    text = re.sub(pattern, 'compensation', text, flags=re.IGNORECASE)
    return text
  

df['raw_text']=df.apply(
      lambda row: clean_html_tags(
        f"{row['job_title']} {row['job_ad_details']}"
    ),
    axis=1
)
# df[["job_id", "raw_text",'y_true']].to_csv("cleaned_job_ads.csv", index=False, encoding="utf-8-sig")


# example_html = df.iloc[1]['job_title'] + df.iloc[1]['job_ad_details']
# print(example_html)


In [ ]:
def parse_salary_string(salary_str):
    pattern = r"(\d+)-(\d+)-([A-Z]+)-([A-Z]+)"
    match = re.match(pattern, salary_str)
    if not match:
        return None  # 无法解析
    
    min_sal, max_sal, _, period = match.groups()
    
    result = {
        "salary_range": f"{min_sal}-{max_sal}",
        "pay_period": period
    }
    
    if min_sal == max_sal:
        result["fix_salary"] = min_sal
    else:
        result["fix_salary"] = None

    return result
  
  
# 新建列来存储拆分后的数据
df['salary_info'] = df['y_true'].apply(parse_salary_string)

# 提取出每个拆分字段
df[['salary_range', 'pay_period', 'fix_salary']] = pd.json_normalize(df['salary_info'])

# 删除原始 salary 列和 salary_info 列（可选）
df.drop(columns=['salary_info'], inplace=True)

# 保存处理后的结果到新的 CSV 文件
df[["job_id", "raw_text",'fix_salary', 'salary_range', 'pay_period', 'y_true']].to_csv('aaa.csv', index=False)